# Homework 1

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Jesse D. Jenkins and Michael R. Davidson (last updated: September 13, 2020)_

This Notebook will walk you through defining a simple transport flow model and then ask you to interact with the solutions and modify to model to add additional constraints...

## Setting up the model

### Load packages

In [ ]:
using JuMP
using Clp
using DataFrames
using CSV

### Define sets

We will define two sets, both as arrays of strings

***Production plants, $P$***

In [ ]:
P=["trenton", "newark"] # production plants

***Markets for products, $M$***

In [ ]:
M=["newyork", "princeton", "philadelphia"] # markets for products

Note that sets can also be defined over intervals (as in `i=1:10`) or numerical vectors (as in `x=[2, 4, 5, 11]`) 

### Define parameters

We'll make use of the defined sets as indexes for our parameters...

***Plant production capacities***

In [ ]:
plants = DataFrame(plant=P, capacity=[350,650])

***Demand for products***

Stored in a [DataFrame](https://juliadata.github.io/DataFrames.jl/stable/)

In [ ]:
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275]
)

A few different ways to index into our DataFrames to access parameters (all of the below are equivalent)

In [ ]:
plants[plants.plant.=="newark",:capacity] # option 1

In [ ]:
plants[plants.plant.=="newark",:].capacity # option 2

In [ ]:
plants.capacity[plants.plant.=="newark"] # option 3

In [ ]:
plants[:,:capacity][plants.plant.=="newark"] # option 4

Note that DataFrame indexing returns an Array by default, in this case, a 1-element Array of type Int64 (64-bit integer), as indicated by `Array{Int64,1}` above. 

To access the single Int64 value, append `[1]` to any of the above to reference the first (and only) element in this array. 

In [ ]:
plants.capacity[plants.plant.=="newark"][1]

In [ ]:
typeof(plants.capacity[plants.plant.=="newark"][1])

In [ ]:
typeof(plants.capacity[plants.plant.=="newark"])

***Distance from plants to markets***

Stored in a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/v0.19/containers/) with data array and symbolic references across each of our sets (plants and markets), converted to Symbols for referencing

In [ ]:
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5;
     0.5 1.5 3.5],
    Symbol.(P),
    Symbol.(M),
)

A couple example references to our DenseAxisArray to access parameters...

In [ ]:
distances[:trenton, :newyork] #example of distance references

In [ ]:
distances[:newark, :newyork] #example of distance references

In [ ]:
distances[Symbol(P[2]),Symbol(M[1])] # another way to find distance from newark to trenton

In [ ]:
distances[Symbol("newark"), Symbol("newyork")] # and a third...

***Costs of transport***

In [ ]:
freight_cost = 90 # Cost of freight shipment per unit of distance

### Create model
(and specify the Clp solver)

In [ ]:
transport = Model(Clp.Optimizer);

### Define variables

***Quantities of product to transport from plant $p \in P$ to market $m \in M$***

In [ ]:
@variable(transport, X[P,M] >= 0)

Example reference to single quantity decision variable, the quantity shipped from Newark to Philadelphia:

In [ ]:
X["newark","philadelphia"]

### Define constraints

***Supply capacity constraint***

In [ ]:
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])

***Demand balance constraint***

Ensure all demand is satisfied at each market

In [ ]:
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])

### Define objective function

Minimize total cost of transport to satisfy all demand.

First we'll define an expression for total cost of shipments...

In [ ]:
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )

Now we'll minimize this total cost

In [ ]:
@objective(transport, Min, eCost)

## Interact with the model

**(a)** Now let's solve the model. In the blank cell below, enter the command for JuMP to solve a model and run the cell

**(b)** You've got a solution. Now query the objective function in the empty cell below and save it to a variable (name of your choice)

**(c)** Now query and save the optimal solution for X (the decisions about shipment quantities from plant to market) to an Array or DataFrame

**(d)** Please interpret your results by writing an explanation in the markdown cell below. 

Which facility or facilities supplies the most demand in New York? Does this result make sense? Why?

Which facility or facilities supplies the most demand in Philadelphia? Does this result make sense? Why?

Which facility or facilities supplies the demand in Princeton? Does this result make sense? Why?

**(d)** A new market in New Brunswick appears, with a demand for 50 units. It is located 1.0 units away from both plants. Add this market to the model and solve again.

**(e)** What is new optimal solution? 

**(f)** Interpret this result in the markdown cell below. Which facility or facilities supplies the demand in New Brunswick? Does this result make sense? Why?